In [ ]:
import sys, subprocess, os
IN_COLAB = 'COLAB_GPU' in os.environ
if IN_COLAB:
    print('Running on Colab: installing dependencies...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet', 'qiskit==1.1.1', 'qiskit-aer==0.14.2', 'stim==1.15.0', 'pymatching==2.2.1', 'numpy==1.26.4', 'scipy==1.12.0', 'matplotlib==3.8.4', 'pandas==2.2.2'])
else:
    print('Not on Colab; ensure you have the repo and dependencies installed (pip install -e .)')


In [ ]:
from surface_code_sim.utils import NoiseParams, ExperimentConfig
from surface_code_sim.qiskit_frontend import sample_syndromes
from surface_code_sim.decoders import LocalDecoder, MwpmDecoder

cfg = ExperimentConfig(
    distance=3,
    rounds=2,
    shots=200,
    noise=NoiseParams(model='depolarizing', p=0.001, readout_error=0.0),
    decoder='mwpm',
    backend='aer',
    seed=0,
)
print(cfg)


In [ ]:
synd = sample_syndromes(cfg)
print('x_meas shape:', synd.x_meas.shape)
print('z_meas shape:', synd.z_meas.shape)
print('x_detection sum:', synd.x_detection.sum())
print('z_detection sum:', synd.z_detection.sum())


In [ ]:
local = LocalDecoder()
mwpm = MwpmDecoder.from_distance(cfg.distance)

local_out = local.decode(synd)
mwpm_out = mwpm.decode(synd)

import numpy as np

def logical_rate(out):
    logical_errors = ((out['x_logical'] | out['z_logical']) != 0).astype(int)
    return float(logical_errors.mean())

print({'decoder': 'local', 'logical_error_rate': logical_rate(local_out)})
print({'decoder': 'mwpm', 'logical_error_rate': logical_rate(mwpm_out)})


In [ ]:
import matplotlib.pyplot as plt

rates = [logical_rate(local_out), logical_rate(mwpm_out)]
labels = ['local', 'mwpm']

plt.bar(labels, rates)
plt.ylabel('Logical error rate')
plt.title('Distance 3, p=0.001, rounds=2, shots=200')
plt.show()
